In [ ]:
# Install required libraries
!pip install -q pandas torch transformers accelerate

# Import libraries and suppress warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import torch
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from google.colab import files

In [ ]:
# Log in to Hugging Face to access your fine-tuned model
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# --- Configuration ---
HF_USERNAME = "your_user_name"
FT_MODEL_NAME = "Qwen2.5-0.5B-DPO-YouTube-Titles"

BASE_MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
YOUR_FT_MODEL_ID = f"{HF_USERNAME}/{FT_MODEL_NAME}"
NUM_IDEAS_TO_TEST = 10 # Number of random ideas to test

In [ ]:
try:
    df_ideas = pd.read_csv('ideas.csv', header=None)
    idea_list = df_ideas[0].tolist()

    if idea_list and idea_list[0].strip().lower() == 'idea':
        idea_list.pop(0)

    # Select a random sample of ideas for testing
    if len(idea_list) < NUM_IDEAS_TO_TEST:
        test_ideas = idea_list
    else:
        random.seed(42)
        test_ideas = random.sample(idea_list, NUM_IDEAS_TO_TEST)
    print(f"Loaded and sampled {len(test_ideas)} ideas for evaluation.")
except FileNotFoundError:
    print("Error: 'ideas.csv' not found. Please make sure you have uploaded it.")

In [ ]:
print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)
base_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

print("Loading fine-tuned model from Hugging Face Hub...")
try:
    ft_model = AutoModelForCausalLM.from_pretrained(YOUR_FT_MODEL_ID)
    ft_tokenizer = AutoTokenizer.from_pretrained(YOUR_FT_MODEL_ID)
    print("Models loaded successfully!")
except Exception as e:
    print(f"Error loading fine-tuned model '{YOUR_FT_MODEL_ID}'.")
    print(f"Error details: {e}")

In [ ]:
from transformers import pipeline

def generate_response(prompt, model, tokenizer, max_length=100, temperature=0.7, device='mps'):
    """
    Generate a response for a given prompt using a Hugging Face text generation pipeline.

    Args:
        prompt (str): Input prompt string.
        model: Fine-tuned language model.
        tokenizer: Corresponding tokenizer.
        max_length (int): Maximum length of the generated response.
        temperature (float): Sampling temperature for generation.
        device (str): Device to run the model on. For example, 'cpu', 'cuda', or 'mps'.

    Returns:
        str: Generated text output.
    """

    generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
    outputs = generator(prompt, max_length=max_length, truncation=True, num_return_sequences=1, temperature=temperature)
    return outputs[0]['generated_text']

In [ ]:
# --- Run Evaluation ---
results = []
print("\nGenerating titles for comparison...")
for i, idea in enumerate(test_ideas):
    print(f"  - Processing idea {i+1}/{len(test_ideas)}: '{idea}'")

    base_title = generate_title(idea, base_model, base_tokenizer)
    ft_title = generate_title(idea, ft_model, ft_tokenizer)

    results.append({
        "idea": idea,
        "base_model_title": base_title,
        "fine_tuned_title": ft_title
    })

df_results = pd.DataFrame(results)

In [ ]:
# --- Display and Download Results ---
print("\n--- Evaluation Results ---")
print(df_results)

# Save results to a CSV file
output_filename = "evaluation_results.csv"
df_results.to_csv(output_filename, index=False)

In [ ]:
import pandas as pd

df = pd.read_csv("evaluation_results.csv")
df.shape

In [ ]:
df.head(10)

In [ ]:
# download the output file
files.download(output_filename)
print(f"\nResults saved and download prompted for '{output_filename}'.")